In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import time

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_20_08_24_v2.xlsx')

In [3]:
df['Abstract']

0      Organic electrochemical transistors are a prom...
1      Organic mixed ionic-electronic conductors (OMI...
2      Organic mixed ionic-electronic conductors (OMI...
3      The field of organic mixed ionic-electronic co...
4      Operational stability underpins the successful...
                             ...                        
118    In recent years, organic electrochemical trans...
119    Organic electrochemical transistors (OECTs) ma...
120    Owing to their potential advantages such as lo...
121    Due to similar transmission characteristics of...
122    Synaptic plasticity functions play a crucial r...
Name: Abstract, Length: 123, dtype: object

In [4]:
GROQ_API_KEY = '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'

In [5]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print('\n', index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    # chat request
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers of the type OMIEC, also known as Organic Mixed Ionic Electronic Conductors."
            },
            {
                "role": "user",
                "content": "For the text I will input next, identify any Organic Mixed Ionic Electronic Conductors (OMIEC) polymers that are used for neuromorphic devices. Provide only polymers that are mentioned in the text and provide only their names, separated by semicolons. If no such polymers are found in the text, the output should be 'None'. Do not repeat any polymer name. Do not output the polymer type. Do not output other types of materials. If the polymer name is written in full, replace it with its abbreviation."

            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=0.3,
        max_tokens=1024,
        top_p=0.9,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    print()
    
    # listas de respostas
    responses_list.append({ 'index': index, 'response': response })

    # tempo total
    total_time += time.time() - start_time
    


 0 of 123 remaining estimated time 0.0
PT; NDI

 1 of 123 remaining estimated time 21.103466749191284
PEDOT:PSS; Mel

 2 of 123 remaining estimated time 25.860033909479775
P3MEEET

 3 of 123 remaining estimated time 28.516488075256348
None

 4 of 123 remaining estimated time 29.861365032196044
None

 5 of 123 remaining estimated time 30.46850609779358
None

 6 of 123 remaining estimated time 30.49908910478864
None

 7 of 123 remaining estimated time 49.2579288482666
None

 8 of 123 remaining estimated time 90.37875096003214
None

 9 of 123 remaining estimated time 124.0158266544342
P-3O; P-4O; P-5O; P-6O

 10 of 123 remaining estimated time 159.57909425822172
PEDOT:PSS

 11 of 123 remaining estimated time 188.49652163187665
None

 12 of 123 remaining estimated time 203.87288563068097
PBBT-Me; PBBT-H; BBL

 13 of 123 remaining estimated time 232.73748738425115
Ru(bpy)(3)(PF6)(2)

 14 of 123 remaining estimated time 249.4922597726186
None

 15 of 123 remaining estimated time 263.4089868

In [6]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/llama3-70B-OMIEC200824_0.3_temp_NEUROMORPHIC_RESPONSES_v2.xlsx', index=False)

#### Playground

In [7]:
df_with_responses

,index,response
0,0,PT; NDI
1,1,PEDOT:PSS; Mel
2,2,P3MEEET
3,3,None
4,4,None
...,...,...
118,118,None
119,119,None
120,120,P(VDF-TrFE); P3HT
121,121,P3HT
